In [1]:
import cv2
import numpy as np
from scipy.ndimage import rotate
from os import listdir
from os.path import isfile, join
from pathlib import Path
import matplotlib.pyplot as plt
from skimage.filters import gaussian
from skimage.segmentation import active_contour

In F:\Anaconda\envs\hdda_hw2\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In F:\Anaconda\envs\hdda_hw2\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In F:\Anaconda\envs\hdda_hw2\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In F:\Anaconda\envs\hdda_hw2\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In F:\Anaconda\envs\hdda_hw2\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_tes

In [2]:
def ToBNW(image_path, fx = 1, ksize = 5):
    image = cv2.imread(image_path)
    # image = cv2.resize(image, dsize = (1224, 918))
    if fx != 1:
        image = cv2.resize(image, None, fx = fx, fy = fx)

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    ## mask of green (36,0,0) ~ (70, 255,255), mask o yellow (15,0,0) ~ (36, 255, 255)
    rgbmask = cv2.inRange(image, (80,50,50), (200, 200, 200))
    rgbmask = cv2.medianBlur(rgbmask, ksize = 3)
    hsv0 = cv2.inRange(hsv, (0,0,50), (100, 250, 250))

    rgb1 = np.copy(rgbmask).astype(np.uint8)
    hsv1 = np.copy(hsv0).astype(np.int)
    hsv2 = np.abs(255 - hsv1)

    combine = np.uint8((hsv2 + rgb1)/2)
    combine[combine < 130] = 0
    mask = cv2.medianBlur(combine, ksize = ksize)
    edge = cv2.Canny(mask, 100, 200)

    return combine

In [3]:
def ToBNW2(fname, fx = 1, ksize = 5): 
    image = cv2.imread(fname)
    
    if fx != 1:
        image = cv2.resize(image, None, fx = fx, fy = fx)
        
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    sat = hsv[:, :, 1]
    sat = cv2.medianBlur(sat, ksize = 3)
    sat1 = cv2.inRange(sat, (0), (60))/255
    rmask = cv2.inRange(image[:, :, 0], (100), (255))/255
#     print(sat1.dtype)
    out = (np.abs(1-sat1) + rmask)*255/2
#     plt.imshow(out, cmap = plt.cm.gray)
#     out = cv2.medianBlur(out, ksize = ksize)
    return np.uint8(out)

In [4]:
def match1(image, temp0, slow=0.7, shigh=1.3, method = cv2.TM_CCORR_NORMED):
    # get flipped image
    best_match_score = 0
    image1 = np.copy(image)
    image1_flip = cv2.flip(image1, 1)
    # change template sizes
    for s in np.arange(slow, shigh, 0.05):
        temp = cv2.resize(temp0, None, fx = s, fy = s)
        # search for angles
        for i in [-3, -2, -1, 0, 1, 2, 3, 15, 16, 17, 18, 19, 20, 21]:
        # for i in range(0, 36):
        #     # the rotate takes angle in degrees
            img_rot = rotate(image1, 10 * i, mode = 'constant', reshape = False, cval = 0)
            img_rotflip = rotate(image1_flip, 10 * i, mode = 'constant', reshape = False, cval = 0)
            res_rot = cv2.matchTemplate(img_rot, temp, method)
            res_rotflip = cv2.matchTemplate(img_rotflip, temp, method)
            if res_rot.max() >= best_match_score:
                best_match_score = res_rot.max()
                res = res_rot
                best_img = img_rot
                info = ('rot', i, s, temp, best_img)
            elif res_rotflip.max() >= best_match_score:
                best_match_score = res_rotflip.max()
                res = res_rotflip
                best_img = img_rotflip
                info = ('rotflip', i, s, temp, best_img)
    return best_match_score, res, info

In [5]:
def rotation1(fname):
    temp1 = ToBNW2('../data/templates/1.png', fx = 1, ksize = 3)
    temp2 = ToBNW2('../data/templates/2.png', fx = 1, ksize = 3)
    temp3 = ToBNW2('../data/templates/3.png', fx = 1, ksize = 3)
    temp4 = ToBNW2('../data/templates/4.png', fx = 1, ksize = 3)

    templst = [temp1, temp2, temp3, temp4]
    image = ToBNW2(fname, fx = 1, ksize = 5)

    H, W = image.shape[0:2]
    # start template matching, and look for different angles:
    best_match_score = 0
    res = None
    # all available methods: methods = ['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR', 'cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']
    method = cv2.TM_CCORR_NORMED

    scorelst, reslst, infolst = [], [], []

    score1, res1, info1 = match1(image, temp1, slow=0.75, shigh=1.3)
    scorelst.append(score1)
    reslst.append(res1)
    infolst.append(info1)
    
    score2, res2, info2 = match1(image, temp2, slow=0.75, shigh=1.3)
    scorelst.append(score2)
    reslst.append(res2)
    infolst.append(info2)
    
    score3, res3, info3 = match1(image, temp3, slow=0.75, shigh=1.2)
    scorelst.append(score3)
    reslst.append(res3)
    infolst.append(info3)
    
    score4, res4, info4 = match1(image, temp3, slow=0.75, shigh=1.2)
    scorelst.append(score4)
    reslst.append(res4)
    infolst.append(info4)

    idx = np.array(scorelst).argmax()
    print(scorelst, idx)
    manual = None
    if scorelst[idx] < 0.91:
        print('This Sample need manual processing! ')
        print('---------')
        manual = fname
    h, w = infolst[idx][3].shape[0:2]

    # plot best image orientation
    best = cv2.imread(fname)
    if infolst[idx][0] == 'rotflip':
        best = cv2.flip(best, 1)
    best = rotate(best, 10 * infolst[idx][1], mode = 'constant', reshape = False, cval = 0)

    loc = np.unravel_index(reslst[idx].argmax(), reslst[idx].shape)

    # get the corresponding window
    cornerr = loc[0]
    cornerc = loc[1]
    window = best[cornerr:cornerr + h, cornerc: cornerc + w, :]

    return best, window, manual

In [ ]:
fname = '../data/CC Lake slice/DSC00902.jpg'
temp1 = ToBNW2('../data/templates/1.png', fx = 1, ksize = 3)
temp2 = ToBNW2('../data/templates/2.png', fx = 1, ksize = 3)
temp3 = ToBNW2('../data/templates/3.png', fx = 1, ksize = 3)
temp4 = ToBNW2('../data/templates/4.png', fx = 1, ksize = 3)
templst = [temp1, temp2, temp3, temp4]
image = ToBNW2(fname, fx = 1, ksize = 5)

H, W = image.shape[0:2]
method = cv2.TM_CCORR_NORMED

scorelst, reslst, infolst = [], [], []
score1, res1, info1 = match1(image, temp1, slow=0.75, shigh=1.25)
scorelst.append(score1)
reslst.append(res1)
infolst.append(info1)

score2, res2, info2 = match1(image, temp2, slow=0.75, shigh=1.25)
scorelst.append(score2)
reslst.append(res2)
infolst.append(info2)

score3, res3, info3 = match1(image, temp3, slow=0.75, shigh=1.25)
scorelst.append(score3)
reslst.append(res3)
infolst.append(info3)

score4, res4, info4 = match1(image, temp3, slow=0.75, shigh=1.25)
scorelst.append(score4)
reslst.append(res4)
infolst.append(info4)

idx = np.array(scorelst).argmax()

loc = np.unravel_index(reslst[idx].argmax(), reslst[idx].shape)
print(loc)
    # print(info[3].shape, 'temp w and h', w, h, 'image W and H', W, H)
    # # # print(info)
    # get the corresponding window


In [ ]:
'''info = rot, i, s, temp, best_img'''
cornerr = loc[0]
cornerc = loc[1]
# image_out = np.copy(image)
# image_out = cv2.circle(image_out, (cornerc, cornerr), radius=5, color=(0, 0, 255), thickness=1)
# image_out = cv2.circle(image_out, (cornerc + w, cornerr + h), radius=5, color=(0, 0, 255), thickness=-1)
print(infolst[idx])

info = infolst[idx]
best = np.copy(image)
s = info[2]
best_temp = info[3]
h, w = best_temp.shape
best_img = info[4]
print(best_temp.shape, best_img.shape)
best_img_overlaid = np.copy(best_img)
best_img_overlaid[cornerr:cornerr+h, cornerc:cornerc+w] = best_temp

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(3, 1)
ax1.imshow(best_temp, cmap=plt.cm.gray)
ax2.imshow(best_img, cmap=plt.cm.gray)
ax3.imshow(best_img_overlaid, cmap=plt.cm.gray)


In [6]:
def test():
    path_oriented = "../data/CC Lake oriented/8/"
    path_extracted = "../data/CC Lake extracted/5/"
    Path(path_oriented).mkdir(parents=True, exist_ok=True)
    Path(path_extracted).mkdir(parents=True, exist_ok=True)
    onlyfiles = [f for f in listdir('../data/CC Lake slice/') if isfile(join('../data/CC Lake slice/', f))]
    # print(onlyfiles)
    examples = ['DSC00896.jpg', 'DSC00897.jpg', 'DSC00914.jpg', 'DSC00922.jpg', 'DSC00949.jpg', 'DSC00959.jpg']
    
    manuallst = []
#     for f in examples[:]:
    for f in onlyfiles:
        fname = join('../data/CC Lake slice/', f)
        print('----Now working on ----')
        print(f)
        best, window, manual = rotation1(fname)
        cv2.imwrite(join(path_oriented, f), best)
        cv2.imwrite(join(path_extracted, f), window)
        if manual is not None:
            manuallst.append(manual)
    print('Please process the following samples manually: ')
    for j in manuallst:
        print(j)

In [7]:
test()

----Now working on ----
DSC00839.JPG
[0.9911255, 0.922699, 0.9102421, 0.9102421] 0
----Now working on ----
DSC00840.JPG
[0.9743048, 0.91854876, 0.9105933, 0.9105933] 0
----Now working on ----
DSC00841.JPG
[0.92944187, 0.9184587, 0.919799, 0.919799] 0
----Now working on ----
DSC00842.JPG
[0.93905073, 0.9469007, 0.9172769, 0.9172769] 1
----Now working on ----
DSC00843.JPG
[0.93180573, 0.96895117, 0.9465903, 0.9465903] 1
----Now working on ----
DSC00844.JPG
[0.9347423, 0.93649465, 0.9583465, 0.9583465] 2
----Now working on ----
DSC00845.JPG
[0.93255913, 0.94879127, 0.9628549, 0.9628549] 2
----Now working on ----
DSC00846.JPG
[0.94002575, 0.90779597, 0.91962856, 0.91962856] 0
----Now working on ----
DSC00847.JPG
[0.9186153, 0.96581215, 0.9399429, 0.9399429] 1
----Now working on ----
DSC00848.JPG
[0.9173293, 0.94962764, 0.93906415, 0.93906415] 1
----Now working on ----
DSC00849.JPG
[0.9365569, 0.91700685, 0.91465634, 0.91465634] 0
----Now working on ----
DSC00850.JPG
[0.9331978, 0.9408403, 

[0.9248425, 0.9637163, 0.9310691, 0.9310691] 1
----Now working on ----
DSC00936.JPG
[0.9158257, 0.9599455, 0.9431031, 0.9431031] 1
----Now working on ----
DSC00937.JPG
[0.93432266, 0.9598584, 0.9285486, 0.9285486] 1
----Now working on ----
DSC00938.JPG
[0.9273196, 0.930871, 0.91611737, 0.91611737] 1
----Now working on ----
DSC00939.JPG
[0.9190613, 0.9620869, 0.91319305, 0.91319305] 1
----Now working on ----
DSC00940.JPG
[0.9319416, 0.93022156, 0.9226748, 0.9226748] 0
----Now working on ----
DSC00941.JPG
[0.9178963, 0.95609164, 0.9354158, 0.9354158] 1
----Now working on ----
DSC00942.JPG
[0.9306729, 0.9457101, 0.9353509, 0.9353509] 1
----Now working on ----
DSC00943.JPG
[0.92772, 0.9611014, 0.9348047, 0.9348047] 1
----Now working on ----
DSC00944.JPG
[0.9256488, 0.93905497, 0.9151184, 0.9151184] 1
----Now working on ----
DSC00945.JPG
[0.92788666, 0.9309861, 0.9246764, 0.9246764] 1
----Now working on ----
DSC00946.JPG
[0.9454534, 0.9242379, 0.91589713, 0.91589713] 0
----Now working on --

In [ ]:
### NOTE
877
896 [0.92752725, 0.9330918, 0.91779184, 0.91779184] 1
897 [0.9302923, 0.938535, 0.91850716, 0.91850716] 1
914 [0.92993504, 0.9326193, 0.9173657, 0.9173657] 1
922
949
959